In [33]:
# Import necessary libraries for data manipulation and machine learning
import pandas as pd
import numpy as np
import torch

In [34]:
# Load the Davis dataset from a PyTorch file
davis_db = torch.load('D:\Drugllm\Drugtargetdata\Davis.pt', weights_only=False)

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Cindy\AppData\Local\Temp\ipykernel_10484\503051307.py:2: SyntaxWarning: invalid escape sequence '\D'
  davis_db = torch.load('D:\Drugllm\Drugtargetdata\Davis.pt', weights_only=False)


In [37]:
# Import additional libraries for machine learning and evaluation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib
from xgboost import XGBRegressor

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize a DataFrame to store evaluation metrics
metrics_df = pd.DataFrame(columns=['Model', 'Dataset', 'RMSE', 'MAE', 'MSE', 'R2'])

def calculate_metrics(y_true, y_pred):
    """Calculate regression metrics."""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, mse, r2


def train_model_predict(
    model_name, model, 
    train_df, val_df, test_df,
    X_train, y_train, X_val, y_val, X_test, y_test,
    save_predictions=True, 
    predictions_dir='D:/Drugllm/predictions',
    save_model=True, 
    model_dir='D:/Drugllm/saved_models',
    datasetname="bind"
):
    """
    Train and evaluate a model, save predictions and pickle model.
    """
    # -------------------------
    # Create directories
    # -------------------------
    os.makedirs(predictions_dir, exist_ok=True)
    os.makedirs(model_dir, exist_ok=True)

    # -------------------------
    # Predictions
    # -------------------------
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    test_pred = model.predict(X_test)

    # -------------------------
    # Save predictions (CSV)
    # -------------------------
    if save_predictions:
        try:
            y_train_arr = np.array(y_train).flatten()
            y_val_arr = np.array(y_val).flatten()
            y_test_arr = np.array(y_test).flatten()

            train_df['Predicted'] = train_pred
            train_df['True'] = y_train_arr
            train_df['Split'] = 'Train'

            val_df['Predicted'] = val_pred
            val_df['True'] = y_val_arr
            val_df['Split'] = 'Validation'

            test_df['Predicted'] = test_pred
            test_df['True'] = y_test_arr
            test_df['Split'] = 'Test'

            predict_df = pd.concat([train_df, val_df, test_df], ignore_index=True)
            predict_df_out = predict_df[['Index_ID', 'Drug_ID', 'Target_ID', 'Affinity', 
                                         'Predicted', 'True', 'Split']]

            safe_name = model_name.replace(" ", "_")
            pred_path = os.path.join(predictions_dir, f'{safe_name}_predictions_{datasetname}.csv')

            predict_df_out.to_csv(pred_path, index=False)

        except Exception as e:
            print(f"Warning: could not save predictions for {model_name}: {e}")

    # -------------------------
    # Save trained model (pickle)
    # -------------------------
    if save_model:
        try:
            safe_name = model_name.replace(" ", "_")
            model_path = os.path.join(model_dir, f"{safe_name}_{datasetname}.pkl")

            with open(model_path, "wb") as f:
                pickle.dump(model, f)

            print(f"🔒 Model saved successfully: {model_path}")

        except Exception as e:
            print(f"Warning: could not save model pickle for {model_name}: {e}")

    return predict_df_out


In [39]:
def prepare_for_psichic(dti_data):
    # Process the DTI dataset to create a simplified DataFrame
    davis_psichic = dti_data[['Index_ID', 'Target', 'Drug']]
    # Rename columns for consistency
    davis_psichic.rename(columns={'Index_ID': 'ID', 'Drug': 'Ligand', 'Target': 'Protein'}, inplace=True)
    # Reorder columns for better readability
    davis_psichic = davis_psichic[['ID', 'Protein', 'Ligand']]
    return davis_psichic

In [40]:
# Set a random seed for reproducibility
SEED = 42

# Create a copy of the Davis dataset for training and testing
dti_data = davis_db.copy().reset_index(drop=True)

dti_data['Index_ID'] = 'Davis' + dti_data.index.astype(str)

# Prepare and save the dataset for Psichic
davis_psichic = prepare_for_psichic(dti_data)
davis_psichic.to_csv('./Psichic_data/davis_db_psichic_input.csv', index=False)

# Split the dataset into training, validation, and test sets
train_df, temp_df = train_test_split(dti_data, test_size=0.3, random_state=SEED)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED)

# Prepare feature matrices and target variables
X_train = np.hstack((np.vstack(train_df['Drug_Features']), np.vstack(train_df['Target_Features'])))
X_val = np.hstack((np.vstack(val_df['Drug_Features']), np.vstack(val_df['Target_Features'])))
X_test = np.hstack((np.vstack(test_df['Drug_Features']), np.vstack(test_df['Target_Features'])))
y_train, y_val, y_test = train_df['Affinity'], val_df['Affinity'], test_df['Affinity']

# Display the shapes of the feature matrices
print(X_train.shape, X_val.shape, X_test.shape)

C:\Users\Cindy\AppData\Local\Temp\ipykernel_10484\2042944623.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  davis_psichic.rename(columns={'Index_ID': 'ID', 'Drug': 'Ligand', 'Target': 'Protein'}, inplace=True)


(18040, 1088) (3866, 1088) (3866, 1088)


In [10]:
dti_data.shape

(25772, 8)

In [13]:
dti_data.loc[1]['Drug_Features'].shape

(768,)

In [14]:
dti_data.loc[1]['Target_Features'].shape

(320,)

In [6]:
dti_data.head()

,Drug_ID,Drug,Target_ID,Target,Y,Affinity,Drug_Features,Target_Features
0,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,AAK1,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,43.0,7.366532,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.009276379, -0.14671455, 0.065141745, 0.037..."
1,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ABL1p,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,10000.0,5.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.052989967, -0.19866328, 0.01483496, 0.0618..."
2,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ABL2,MVLGTVLLPPNSYGRDQDTSLCCLCTEASESALPDLTDHFASCVED...,10000.0,5.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[-0.028185444, -0.17497379, 0.061383072, 0.063..."
3,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ACVR1,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,10000.0,5.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[0.060061518, -0.12135677, 0.12382316, 0.06401..."
4,11314340,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12,ACVR2A,MGAAAKLAFAVFLISCSSGAILGRSETQECLFFNANWEKDRTNQTG...,10000.0,5.000000,"[0.7605508, 0.46918112, 0.022348084, -0.290485...","[0.10380349, -0.11144593, 0.11350971, 0.110970..."


In [26]:
# Scale the feature matrices for models that require normalized input
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [27]:
# Train and evaluate a Random Forest model
print('Random Forest')
rf_model = RandomForestRegressor(n_estimators=100, random_state=SEED)
rf_model.fit(X_train_scaled, y_train)
predict_rf = train_model_predict("Random Forest", rf_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")

<>:5: SyntaxWarning: invalid escape sequence '\G'
<>:5: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Cindy\AppData\Local\Temp\ipykernel_10484\3418657126.py:5: SyntaxWarning: invalid escape sequence '\G'
  predict_rf = train_model_predict("Random Forest", rf_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")


Random Forest


In [28]:
# Train and evaluate an SVR model
print('SVR')
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train_scaled, y_train)
predict_rf = train_model_predict("SVR", svr_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")

<>:5: SyntaxWarning: invalid escape sequence '\G'
<>:5: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Cindy\AppData\Local\Temp\ipykernel_10484\1848316792.py:5: SyntaxWarning: invalid escape sequence '\G'
  predict_rf = train_model_predict("SVR", svr_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")


SVR


In [29]:
# Train and evaluate a Gradient Boosting Machine (GBM) model
print('GBM')
gbm_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=SEED)
gbm_model.fit(X_train, y_train)
predict_rf = train_model_predict("GBM", gbm_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")

<>:5: SyntaxWarning: invalid escape sequence '\G'
<>:5: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Cindy\AppData\Local\Temp\ipykernel_10484\3332188143.py:5: SyntaxWarning: invalid escape sequence '\G'
  predict_rf = train_model_predict("GBM", gbm_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")


GBM


In [30]:
# Train and evaluate a Linear Regression model
print('Linear Regression')
lin_reg_model = LinearRegression()
lin_reg_model.fit(X_train_scaled, y_train)
predict_rf = train_model_predict("Linear Regression", lin_reg_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")

<>:5: SyntaxWarning: invalid escape sequence '\G'
<>:5: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Cindy\AppData\Local\Temp\ipykernel_10484\2818403707.py:5: SyntaxWarning: invalid escape sequence '\G'
  predict_rf = train_model_predict("Linear Regression", lin_reg_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")


Linear Regression


In [31]:
# Train and evaluate a Multi-Layer Perceptron (MLP) model
print('MLP')
mlp_model = MLPRegressor(hidden_layer_sizes=(512, 256), activation='relu', max_iter=200, random_state=SEED)
mlp_model.fit(X_train_scaled, y_train)
predict_rf = train_model_predict("MLP", mlp_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")

<>:5: SyntaxWarning: invalid escape sequence '\G'
<>:5: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Cindy\AppData\Local\Temp\ipykernel_10484\2214724377.py:5: SyntaxWarning: invalid escape sequence '\G'
  predict_rf = train_model_predict("MLP", mlp_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")


MLP


In [32]:
# Train and evaluate an XGBoost model
print('XGBoost')
xgb_model = XGBRegressor(random_state=SEED, eval_metric='rmse')
xgb_model.fit(X_train, y_train)
predict_rf = train_model_predict("XGBoost", xgb_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")

<>:5: SyntaxWarning: invalid escape sequence '\G'
<>:5: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Cindy\AppData\Local\Temp\ipykernel_10484\1058422433.py:5: SyntaxWarning: invalid escape sequence '\G'
  predict_rf = train_model_predict("XGBoost", xgb_model, train_df, val_df, test_df, X_train, y_train, X_val, y_val, X_test, y_test, save_predictions=True, predictions_dir='D:\Github\llm-drug-agent\Prediction_Results',datasetname="davis")


XGBoost
